In [1]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['font.sans-serif'] = ['SimHei'] # 正常显示汉字
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

# 1.演示DataFrame加载指定行列的数据

## 1.1演示df加载：列

In [ ]:
df = pd.read_csv('mdata/gapminder.tsv', sep='\t')
df.head()

In [ ]:
# 查看一下基本信息
df.shape
df.dtypes
df.info()
df.describe(include='all')

In [ ]:
# 加载指定列的数据
df['country']

## 1.2 df加载：行

In [ ]:
df.loc[0] # 根据行索引获取数据
df.iloc[0]  # 根据行号获取

In [ ]:
df.tail(n=1)
df.tail(1)
df.head(1)

## 1.3 df加载：指定行、列数据

In [ ]:
# 格式 :df.loc[[行],[列]] 行索引+列名
# df.iloc[[行],[列]] 行号+列的编号

df.loc[[0,1,2]] # 获取行索引为0,1,2的数据
df.iloc[[0,99,999]] # 获取行号为 0,99,999的数据

# 加载指定行和指定列
df.loc[[20,21,22],['country','year','pop']]
df.iloc[[20,21,22],[0,2,4]]

In [ ]:
df.loc[:,['country']]
df.loc[:,['country','continent','lifeExp']]
df.iloc[:,[0]]
df.iloc[:,[0,1,3]]
df.iloc[:,range(3)]
df.iloc[:,:3]
df.iloc[:,range(1,5,2)]
df.iloc[:,1:5:2]

# 2 DataFrame对象-分组聚合

In [ ]:
# 查看源数据
df.head()

In [ ]:
# 分组统计格式:df.groupby('分组字段')['聚合操作的字段'].具体的聚合函数()
# 分组统计格式:df.groupby(['分组字段1','分组字段2'])['聚合操作的字段1','聚合操作的字段2'].具体的聚合函数名()

# 2.基于上市的格式、完成如下的需求
## 需求1： 统计每一年平均寿命、平均人口、平均gdp
df.groupby('year')['lifeExp'].mean()
df.groupby('year')['pop'].mean()
df.groupby('year')['gdpPercap'].mean()

In [ ]:
## 需求2：按照大洲进行计算，每个大洲的平均预期寿命，平均人口、平均gdp
df.groupby(['year','continent'])[['lifeExp','pop','gdpPercap']].mean()
# agg函数用于多字段的不同聚合需求
# 格式:agg({'列名':'聚合函数名','列名':'聚合函数名'})
df.groupby(['year','continent']).agg({'lifeExp':'mean','pop':'max','gdpPercap':'min'})


In [ ]:
# 需求3，每个大洲列出了多少个国家和地区
df.groupby('continent')['country'].value_counts() # 每个国家杯统计的次数
df.groupby('continent')['country'].nunique()

In [ ]:
# 需求4：使用自定义函数，完成计算平均值
def my_mean(col):
    return col.sum()/col.size

df.groupby('continent').lifeExp.agg(my_mean) # 不要加小括号

In [ ]:
# 需求5 ：计算全球平均预期寿命的平均值 和分组之后平均值差值
def my_mean_diff(col,diff_val):
    return col.mean() - diff_val

gm = df.lifeExp.mean()
df.groupby('continent').lifeExp.agg(my_mean_diff,diff_val = gm)

In [ ]:
# 需求6：agg（）函数，同时传入多个函数
# df.groupby('year').lifeExp.agg([np.count_nonzero,np.mean,np.std])

df.groupby('year').agg({'lifeExp': 'mean','pop':'max','gdpPercap':'min'})

## 2.2 分组转换

In [ ]:
# 2.自定义函数，计算 标准分数 
def my_zscore(x):
    return x - x.mean() /x.std()

# 按年分组，计算lifeExp列zscore的分数
df.groupby('year').lifeExp.transform(my_zscore)


In [ ]:
# 需求2：按照性别分组，计算男女
# sample :随机取样的，参1：取几个值，参2：随机种子，如果种子一样，每次获取的结果是一样的
tips_10 = pd.read_csv('mdata/tips.csv').sample(10,random_state=20)
tips_10

In [ ]:
# 人为给total_bill创建几个空值
# np.random.permutation(tips_10.index)根据tips_10这个df对象的索引，对其进行随机打散
tips_10.loc[np.random.permutation(tips_10.index)[:4] ,'total_bill'] =np.nan

tips_10
# 查看各列的非空值统计情况


In [ ]:

def my_fillna(col):
    return col.fillna(col.mean())

tips_10.groupby('sex').total_bill.transform(my_fillna)

## 2.3 分组过滤

In [ ]:
tips = pd.read_csv('mdata/tips.csv')
tips

tips['size'].value_counts()
tmp_df = tips.groupby('size').filter(lambda x:x['size'].count()>10)
tmp_df['size'].value_counts()

## 2.4 分组对象 DataFrameGroupBy演示

In [ ]:
df = pd.read_csv('mdata/tips.csv').sample(10,random_state=20)
df
# 根据性别分组,获取分组对象


In [ ]:
# 根据性别获取分组对象
grouped = df.groupby('sex')
# 通过groups属性查看计算过的分类
grouped.groups



# 通过 get_grouped获取分组
grouped.get_group('Male')

for sex_group in grouped:
    print(sex_group)

# 需求：按性别和用餐时间分组，计算小费数据平均值

df.groupby(['sex','time']).tip.mean()

# 3.DataFrame基本绘图

In [ ]:
df.groupby('year')['lifeExp'].mean().plot( figsize=(10,5)) # 默认折线图
df.groupby('year')['lifeExp'].mean().plot(kind = 'bar',figsize=(20,10)) # 直方图
# figsize =（宽、高）

# 4. DataFrame对象，常用的统计值的方法

In [ ]:
movie = pd.read_csv('movie.csv')
movie.head()
movie.shape
movie.dtypes
movie.info() # 每列的基本信息，非空值数量，数据类型
movie.describe() # 只统计数值列的信息
movie.describe(include='object')

# 5. DataFrame对象，常见的排序方式

In [ ]:
movie.columns
movie.head() # 查看前5条数据，看看数据格式


In [ ]:
# 完成如下的需求
# 需求1：找到小成本，高口碑的电影
movie2 = movie[['movie_title','budget','imdb_score']]
movie2.head()
# 找出评分最高的100部电影
movie2.nlargest(100,'imdb_score')
movie2.nlargest(100,'imdb_score').nsmallest(10,'budget')


In [ ]:
# 需求2： 找到每年imdb_score评分最高的电影

movie3 = movie[['movie_title','title_year','imdb_score']]
movie3.head()

# 按照年、评分排序
movie3.sort_values(['title_year','imdb_score'],ascending = [False,False])
movie3.sort_values(['title_year','imdb_score'],ascending = False) # 效果同上

# 基于上述的数据，去获取每年的第一条数据结果
# subset:表示子集，即参考某列的值进行去重删除
movie3.sort_values(['title_year','imdb_score'],ascending=False).drop_duplicates(subset = 'title_year')

# 6.综合小案例--链家数据分析

In [25]:
# 加载数据
df = pd.read_csv('mdata/LJdata.csv')

In [ ]:
# 将上述的列名改成英文
df.columns = ['district','address','title','house_type','area','price','floor','build_time','direction','update_time','view_num','extra_info','link']
df.head()

In [ ]:
df.shape
df.info()
df.describe()

In [ ]:
# 完成一下需求
# 需求1 找到租金最低、最高的房子
df.sort_values('price').head(1).price
df.sort_values('price',ascending= False).head(1).price
df[df.price==df.price.max()]
df[df.price==df.price.min()]


In [ ]:
# 需求2 找到最近新上的十套房源
df.sort_values('update_time',ascending=False).head(10)


In [ ]:
# 需求3 查看所有更新时间
df['update_time'].unique()


In [ ]:
# 需求4 不同看房人数的房源数量
df.view_num.mean()
df.view_num.median()
tmp_df = df.groupby('view_num',as_index=False)['district'].count() 
tmp_df.columns = ['view_num','count']
tmp_df


In [ ]:
# 需求5 把上述的count绘制图形
print(tmp_df[['count']]) # dataframe对象
print(type(tmp_df[['count']]))
print(tmp_df['count']) # series对象
print(type(tmp_df['count']))

tmp_df[['count']].plot(kind='bar' , figsize=(20,10))
plt.show()


In [ ]:
# 需求6 房租的价格分布
df['price'].mean()
df['price'].max()
df['price'].min()


In [ ]:
# 需求7 看房人数最多的朝向
tmp_df = df.groupby('direction',as_index=False)['view_num'].sum()
tmp_df[tmp_df.view_num == tmp_df.view_num.max()]


In [ ]:
# 需求8 房型分布情况，绘图
df.groupby('house_type',as_index= False)['district'].count()
df.groupby('house_type')['district'].count().plot(kind = 'bar', figsize=(20,10))


In [ ]:
# 需求9: 最受欢迎的房型
tmp_df = df.groupby('house_type',as_index=False)['view_num'].sum()
tmp_df[tmp_df.view_num == tmp_df.view_num.max()]


In [ ]:
# 需求10
df['price_per_m2'] = df['price'] / df['area']
df.head()
df.price_per_m2.mean()


In [ ]:
# 需求11
df[['address','view_num']].groupby('address').view_num.sum()
tmp_df = df[['address','view_num']].groupby('address',as_index = False).agg({'view_num':'sum'})
tmp_df.sort_values('view_num',ascending= False)
tmp_df.nlargest(10,'view_num')


In [ ]:
# 需求12 出租房源最多的小区
tempdf = df.groupby('address',as_index=False).district.count()
tempdf.columns = ['address','count']
tempdf.nlargest(1,'count')
tempdf.sort_values('count').tail(1)